# Trace Exploration 

In [1]:
%matplotlib inline

In [2]:
import os
import sys
print(sys.version)

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.collections import LineCollection

import ipywidgets as widgets

3.10.8 (main, Nov  1 2022, 14:18:21) [GCC 12.2.0]


In [3]:
sys.path.append("./SERVER/deployment/")

In [4]:
from config import FEATURE_VECTOR_MAP, OUTPUT_VECTOR, OUTPUT_VECTOR_RANGES

In [5]:
plt.rcParams["figure.figsize"] = (15, 5)
plt.style.use("dark_background")
cmap = colormaps.get_cmap("rainbow")

### Load data 

In [6]:
sessions = sorted(Path("./SERVER/sessions/").glob("Session*"))
root = sessions[-1]
save_path = Path("./EXPERIENCE_POLAROIDS")/root.parts[-1]
save_path.mkdir(parents=True, exist_ok=True)

In [7]:
print(root)

SERVER/sessions/Session_20230325_132110


In [8]:
fps = root.glob("*.csv")
esps = sorted([x.parts[-1][:-4] for x in fps])
df = None

@widgets.interact
def selectESP(headset=esps):
    global df
    if headset is None:
        print("No recorded ESPs in session", root)
        return
        
    df = pd.read_csv(root/(headset+".csv"))
    if len(df) == 0:
        print("No data recorded for", headset)
        return
    
    df["time"] = pd.to_datetime(df["time"], unit='s')
    esp = int(headset[-1])
    
    fig, axes = plt.subplots(3, figsize=(8, 8), sharex=False, dpi=200)
    for i, f in enumerate(FEATURE_VECTOR_MAP):
        y = df[f]
        y = (y - y.min()) / (y.max() - y.min())
        c = cmap(i/len(FEATURE_VECTOR_MAP))
        if f[0] == 'h':
            f = "heart:delta"
        axes[0].plot(df.time, y, label=f, lw=2)
    axes[0].set_yticks([])
    axes[0].set_ylabel("sensor data", fontsize=14)
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=4, framealpha=1.0)
    axes[0].set_xlim(df["time"].iloc[0], df["time"].iloc[-1])
        
    for i, f in enumerate(OUTPUT_VECTOR):
        y = df[f]
        y = (y - y.min()) / (y.max() - y.min())
        c = cmap(i/len(FEATURE_VECTOR_MAP))
        axes[1].plot(df.time, y, label=f, lw=2)  
    axes[1].set_yticks([])
    axes[1].set_ylabel("stimulus", fontsize=14)
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, 1.1), ncol=6, framealpha=1.0)
    axes[1].set_xlim(df["time"].iloc[0], df["time"].iloc[-1])
    
    points = np.array(df[["map_x", "map_y"]]).reshape(-1, 1, 2)
    segs = np.concatenate([points[:-1], points[1:]], axis=1)
    alphas = np.linspace(0.0, 1.0, len(segs))
    lc = LineCollection(segments=segs, alpha=0.5, lw=3, color=f'C{esp}')
    axes[2].add_collection(lc)
    axes[2].set_xlim(-1, 1)
    axes[2].set_ylim(-1, 1)
    axes[2].set_yticks([]); axes[2].set_xticks([])
    axes[2].set_ylabel("map", fontsize=14)
    
    fig.autofmt_xdate()
    plt.suptitle(f"Experience of {headset}", fontsize=16)
    plt.tight_layout()
    plt.savefig(save_path/(headset+".png"))
    plt.show()

interactive(children=(Dropdown(description='headset', options=('ESP0', 'ESP3', 'ESP5', 'ESP6', 'ESP7'), value=…